# RE benchmark
infer relations, plus graph inference: REBEL, OpenNRE, qwikidata, etc.
  * ThuNLP `OpenNRE`
  * Babelscape `REBEL`
  * ZS-BERT

In [1]:
import json

import spacy
from zshot import PipelineConfig, displacy
from zshot.utils.data_models import Entity, Relation
from zshot.relation_extractor import RelationsExtractorZSRC
from zshot.mentions_extractor import MentionsExtractorSpacy
from zshot.linker import LinkerRegen

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch

In [94]:
with open("../data/wiki_guardians.json", "r") as fh:
    text: str = json.load(fh)["text"]
    paragraph = text.split("\n\n\n")[0]

In [16]:
start = paragraph.split("\n")[0]

In [66]:
start

"Guardians of the Galaxy Vol. 3 (stylized in marketing as Guardians of the Galaxy Volume 3) is a 2023 American superhero film based on the Marvel Comics superhero team Guardians of the Galaxy, produced by Marvel Studios, and distributed by Walt Disney Studios Motion Pictures. It is the sequel to Guardians of the Galaxy (2014) and Guardians of the Galaxy Vol. 2 (2017), and the 32nd film in the Marvel Cinematic Universe (MCU). Written and directed by James Gunn, it features an ensemble cast including Chris Pratt, Zoe Saldaña, Dave Bautista, Karen Gillan, Pom Klementieff, Vin Diesel, Bradley Cooper, Will Poulter, Sean Gunn, Chukwudi Iwuji, Linda Cardellini, Nathan Fillion, and Sylvester Stallone. In the film, the Guardians must save Rocket's (Cooper) life from the High Evolutionary (Iwuji)."

## Hello world examples

In [3]:
import IPython
import spacy
from zshot import PipelineConfig, displacy
from zshot.linker import Linker, LinkerSMXM
from zshot.utils.data_models import Entity

config = PipelineConfig(
 entities=[
 Entity(name="company", description="A company, abbreviated as co., is a legal entity representing an association of people, whether natural, legal or a mixture of both, with a specific objective. Company members share a common purpose and unite to achieve specific, declared goals."),
 Entity(name="chemical compound", description="Any of a large class of chemical compounds in which one or more \
  atoms of carbon are covalently linked to atoms of other elements, most commonly hydrogen, oxygen, or nitrogen")
 ], 
 linker=LinkerSMXM()
)
nlp = spacy.blank("en")
nlp.add_pipe("zshot", config=config, last=True)

text = "Acetamide (systematic name: ethanamide) is an organic compound with the formula CH3CONH2. It is the simplest amide derived from acetic acid. It finds some use as a plasticizer and as an industrial solvent."
doc = nlp(text)

/Users/louis.guitton/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [4]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [5]:
import spacy
from zshot import PipelineConfig, displacy
from zshot.linker import LinkerSMXM
from zshot.utils.data_models import Entity, Relation
from zshot.relation_extractor import RelationsExtractorZSRC

config = PipelineConfig(
 entities=[
    Entity(name="company", description="The name of a company"),
    Entity(name="location", description="A physical location"),
    Entity(name="chemical compound", description="Any of a large class of chemical compounds in which one or more \
           atoms of carbon are covalently linked to atoms of other elements, most commonly hydrogen, oxygen, or nitrogen")
 ], 
 relations=[
    Relation(name="similar to", description="Is similar to"),
    Relation(name="parent", description="Is the parent of someone")
 ], 
 linker=LinkerSMXM(),
 relations_extractor=RelationsExtractorZSRC(thr=0.8),
)
nlp = spacy.blank("en")
nlp.add_pipe("zshot", config=config, last=True)

text = "CH2O2 is a chemical compound similar to Acetamide used in International \
Business Machines Corporation (IBM) to create new materials that act like PAGs."
doc = nlp(text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
classifying documents: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:03<00:00,  3.86s/it]


In [6]:
displacy.render(doc, style="rel", jupyter=True)

Getting this error
```
File ~/workspace/mlops-talk-llm-kg/venv/lib/python3.11/site-packages/transformers/generation/logits_process.py:1235, in PrefixConstrainedLogitsProcessor.__call__(self, input_ids, scores)
   1233         prefix_allowed_tokens = self._prefix_allowed_tokens_fn(batch_id, sent)
   1234         if len(prefix_allowed_tokens) == 0:
-> 1235             raise ValueError(
   1236                 f"`prefix_allowed_tokens_fn` returned an empty list for batch ID {batch_id}."
   1237                 f"This means that the constraint is unsatisfiable. Please check your implementation"
   1238                 f"of `prefix_allowed_tokens_fn` "
   1239             )
   1240         mask[batch_id * self._num_beams + beam_id, prefix_allowed_tokens] = 0
   1242 return scores + mask

ValueError: `prefix_allowed_tokens_fn` returned an empty list for batch ID 0.This means that the constraint is unsatisfiable. Please check your implementationof `prefix_allowed_tokens_fn` 
```
with `transformers==4.37.2`, I will try to roll back to the version in the Google Colab `transformers==4.35.2`

In [8]:
nlp = spacy.load('en_core_web_sm')

# zero shot definition of entities
nlp_config = PipelineConfig(
    mentions_extractor=MentionsExtractorSpacy(),
    linker=LinkerRegen(),
    entities=[
        Entity(name='Paris',
               description='Paris is located in northern central France, in a north-bending arc of the river Seine'),
        Entity(name='IBM',
               description='International Business Machines Corporation (IBM) is an American multinational technology corporation headquartered in Armonk, New York'),
        Entity(name='New York', description='New York is a city in U.S. state'),
        Entity(name='Florida', description='southeasternmost U.S. state'),
        Entity(name='American',
              description='American, something of, from, or related to the United States of America, commonly known as the United States or America'),
        Entity(name='Chemical formula',
               description='In chemistry, a chemical formula is a way of presenting information about the chemical proportions of atoms that constitute a particular chemical compound or molecul'),
        Entity(name='Acetamide',
               description='Acetamide (systematic name: ethanamide) is an organic compound with the formula CH3CONH2. It is the simplest amide derived from acetic acid. It finds some use as a plasticizer and as an industrial solvent.'),
        Entity(name='Armonk',
               description='Armonk is a hamlet and census-designated place (CDP) in the town of North Castle, located in Westchester County, New York, United States.'),
        Entity(name='Acetic Acid',
               description='Acetic acid, systematically named ethanoic acid, is an acidic, colourless liquid and organic compound with the chemical formula CH3COOH'),
        Entity(name='Industrial solvent',
               description='Acetamide (systematic name: ethanamide) is an organic compound with the formula CH3CONH2. It is the simplest amide derived from acetic acid. It finds some use as a plasticizer and as an industrial solvent.'),
    ]
)
nlp.add_pipe('zshot', config=nlp_config, last=True)

text = 'International Business Machines Corporation (IBM) is an American multinational technology corporation' \
        ' headquartered in Armonk, New York, with operations in over 171 countries.'

doc = nlp(text)
displacy.render(doc, style="ent", jupyter=True)

## Custom example

In [30]:
nlp = spacy.load("en_core_web_sm")
sents = list(nlp(paragraph).sents)

In [36]:
nlp = spacy.load("en_core_web_sm")

nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-mbert-base-multinerd"})

config = PipelineConfig(
    mentions_extractor = MentionsExtractorSpacy(),
    entities=[
        Entity(name="PERSON", description="People, including fictional"),
        Entity(name="WORK OF ART", description="Titles of books, songs, etc."),
        Entity(name="ORGANIZATION", description="Companies, agencies, institutions, organizations, etc.")
    ],
    linker=LinkerRegen(),
    relations=[
        Relation(name="takes place in fictional universe", description="the subject is a work describing a fictional universe, i.e. whose plot occurs in this universe"), # https://www.wikidata.org/wiki/Property:P1434
        Relation(name="present in work", description="this (fictional or fictionalized) entity, place, or person appears in that work as part of the narration"), # https://www.wikidata.org/wiki/Property:P1441
        Relation(name="performer", description="actor, musician, band or other performer associated with this role or musical work"), # https://www.wikidata.org/wiki/Property:P175
        Relation(name="director", description="director(s) of film, TV-series, stageplay, video game or similar"), # https://www.wikidata.org/wiki/Property:P57
        Relation(name="followed by", description="immediately following item in a series of which the subject is a part"), # https://www.wikidata.org/wiki/Property:P156
    ], 
    relations_extractor=RelationsExtractorZSRC(thr=0.8),
)
nlp.add_pipe("zshot", config=config, last=True)

In [63]:
from spacy.language import Language

re = RelationsExtractorZSRC(thr=0.8)
re.set_relations([
    Relation(name="takes place in fictional universe", description="the subject is a work describing a fictional universe, i.e. whose plot occurs in this universe"), # https://www.wikidata.org/wiki/Property:P1434
    Relation(name="present in work", description="this (fictional or fictionalized) entity, place, or person appears in that work as part of the narration"), # https://www.wikidata.org/wiki/Property:P1441
    Relation(name="performer", description="actor, musician, band or other performer associated with this role or musical work"), # https://www.wikidata.org/wiki/Property:P175
    Relation(name="director", description="director(s) of film, TV-series, stageplay, video game or similar"), # https://www.wikidata.org/wiki/Property:P57
    Relation(name="followed by", description="immediately following item in a series of which the subject is a part"), # https://www.wikidata.org/wiki/Property:P156
])

@Language.component("zshot_simple")
def my_zshot(doc):
    doc._.spans = doc.ents
    re.extract_relations([doc])
    return doc

In [64]:
nlp = spacy.load("en_core_web_sm")

nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-mbert-base-multinerd"})
nlp.add_pipe("zshot_simple")

<function __main__.my_zshot(doc)>

## REBEL 
Ref: https://huggingface.co/spaces/ml6team/Knowledge-graphs/blob/main/rebel.py

In [67]:
from typing import List
from transformers import pipeline
from pyvis.network import Network
from functools import lru_cache
import spacy


DEFAULT_LABEL_COLORS = {
    "ORG": "#7aecec",
    "PRODUCT": "#bfeeb7",
    "GPE": "#feca74",
    "LOC": "#ff9561",
    "PERSON": "#aa9cfc",
    "NORP": "#c887fb",
    "FACILITY": "#9cc9cc",
    "EVENT": "#ffeb80",
    "LAW": "#ff8197",
    "LANGUAGE": "#ff8197",
    "WORK_OF_ART": "#f0d0ff",
    "DATE": "#bfe1d9",
    "TIME": "#bfe1d9",
    "MONEY": "#e4e7d2",
    "QUANTITY": "#e4e7d2",
    "ORDINAL": "#e4e7d2",
    "CARDINAL": "#e4e7d2",
    "PERCENT": "#e4e7d2",
}

@lru_cache(maxsize=16)
def generate_partial_graph(text: str):
    triplet_extractor = pipeline(
        'text2text-generation',
        model='Babelscape/rebel-large',
        tokenizer='Babelscape/rebel-large'
    )

    triples = triplet_extractor(
        text,
        return_tensors=True,
        return_text=False)

    if len(triples) == 0:
        return []

    a = [triples[0]["generated_token_ids"]]

    extracted_text = triplet_extractor.tokenizer.batch_decode(a)
    extracted_triplets = extract_triplets(extracted_text[0])
    return extracted_triplets


def extract_triplets(text):
    """
    Function to parse the generated text and extract the triplets
    """
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append(
                    {'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append(
                    {'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append(
            {'head': subject.strip(), 'type': relation.strip(), 'tail': object_.strip()})

    return triplets


In [73]:
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("span_marker", config={"model": "tomaarsen/span-marker-mbert-base-multinerd"})

In [105]:
texts = text.replace("\n\n\n", "\n").replace("\n\n", "\n").split("\n")[:5]

In [106]:
doc = nlp("\n".join(texts).lower())
NERs = [ent.text for ent in doc.ents]
NER_types = [ent.label_ for ent in doc.ents]

triplets = []
for _text in texts:
    triplets.extend(generate_partial_graph(_text))
heads = [t["head"].lower() for t in triplets]
tails = [t["tail"].lower() for t in triplets]

nodes = list(set(heads + tails))
net = Network(directed=True, width="700px", height="700px", notebook=True)

for n in nodes:
    if n in NERs:
        NER_type = NER_types[NERs.index(n)]
        if NER_type in NER_types:
            if NER_type in DEFAULT_LABEL_COLORS.keys():
                color = DEFAULT_LABEL_COLORS[NER_type]
            else:
                color = "#666666"
            net.add_node(n, title=NER_type, shape="circle", color=color)
        else:
            net.add_node(n, shape="circle")
    else:
        net.add_node(n, shape="circle")

unique_triplets = set()
def stringify_trip(x): return x["tail"] + x["head"] + x["type"].lower()
for triplet in triplets:
    if stringify_trip(triplet) not in unique_triplets:
        net.add_edge(triplet["head"].lower(), triplet["tail"].lower(),
                     title=triplet["type"], label=triplet["type"])
        unique_triplets.add(stringify_trip(triplet))

net.repulsion(
    node_distance=200,
    central_gravity=0.2,
    spring_length=200,
    spring_strength=0.05,
    damping=0.09
)
net.set_edge_smooth('dynamic')
net.show("example.html")

example.html


TODO:
- ensure chunking so that length is below max input length to not get index error
- run it on the entire page
- compare the [ML6 implementation](https://huggingface.co/spaces/ml6team/Knowledge-graphs/blob/main/rebel.py) with [the huggingface model card implementation](https://huggingface.co/Babelscape/rebel-large) below 
- adapt the [multilingual model card implementation](https://huggingface.co/Babelscape/mrebel-large) to a more readable implementation, but simpler than textgraphs
- use together with SpanMarker to create graph
- think about enforcing controlled vocabulary like in zshot

In [ ]:
from transformers import pipeline

triplet_extractor = pipeline('text2text-generation', model='Babelscape/rebel-large', tokenizer='Babelscape/rebel-large')
# We need to use the tokenizer manually since we need special tokens.
extracted_text = triplet_extractor.tokenizer.batch_decode([triplet_extractor("Punta Cana is a resort town in the municipality of Higuey, in La Altagracia Province, the eastern most province of the Dominican Republic", return_tensors=True, return_text=False)[0]["generated_token_ids"]])
print(extracted_text[0])
# Function to parse the generated text and extract the triplets
def extract_triplets(text):
    triplets = []
    relation, subject, relation, object_ = '', '', '', ''
    text = text.strip()
    current = 'x'
    for token in text.replace("<s>", "").replace("<pad>", "").replace("</s>", "").split():
        if token == "<triplet>":
            current = 't'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
                relation = ''
            subject = ''
        elif token == "<subj>":
            current = 's'
            if relation != '':
                triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
            object_ = ''
        elif token == "<obj>":
            current = 'o'
            relation = ''
        else:
            if current == 't':
                subject += ' ' + token
            elif current == 's':
                object_ += ' ' + token
            elif current == 'o':
                relation += ' ' + token
    if subject != '' and relation != '' and object_ != '':
        triplets.append({'head': subject.strip(), 'type': relation.strip(),'tail': object_.strip()})
    return triplets
extracted_triplets = extract_triplets(extracted_text[0])
print(extracted_triplets)
